# Modelo detección de fraude biométrico en proceso de onboarding digital

## Cuaderno de cálculo de indicadores cruzados con modelos individuales

### Descripción:
Este cuaderno permite probar la capacidad cruzada de los modelos individuales, los resultados se usan para determinar el orden óptimo de ejecución

#### Dependencias 

In [1]:
from PIL import Image

import sys
sys.path.append('../src/')  

from CLASE_0 import FaceVerifier
from CLASE_I import AgeChecker
from CLASE_II_IV import Fake_II_IV_detector
from CLASE_III_A  import Fake_III_A_detector
from CLASE_III_B  import Fake_III_B_detector


import matplotlib.pyplot as plt

import os
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from tqdm import tqdm

import numpy as np
from gurobipy import Model, GRB

import warnings
warnings.filterwarnings('ignore')

#### Rutas a modelos

In [2]:
paths = {
    'predictor_path'        : "../modelos/shape_predictor_68_face_landmarks.dat",
    'face_rec_model_path'   : "../modelos/dlib_face_recognition_resnet_model_v1.dat",
    'model_weights_path'    : "../modelos/weights.pt",
    
    'model_II_IV_path'      : "../modelos/CLASE_II_IV_Random_forest.joblib",
    'model_III_A_path'      : "../modelos/CLASE_III_A_CNN.h5",
    'model_III_B_path'      : "../modelos/CLASE_III_B_CNN.h5",



    'imagenes_ID'           :"../data/train/base_15k/ID/ID",
    'imagenes_VAL'          :"../data/train/base_15k/VAL/VAL",

    'imagenes_ID_I_False'   :"../data/train/CLASE I, EDAD/ID_Adultos",    
    'imagenes_ID_I_True'    :"../data/train/CLASE I, EDAD/ID_Menores_RevCM", 

    'imagenes_ID_II_False'  :"../data/train/CLASE II y IV, FAKE_COLOR/ID_REAL/ID",    
    'imagenes_ID_II_True'   :"../data/train/CLASE II y IV, FAKE_COLOR/ID_FAKE_II/ID", 

    'imagenes_ID_IV_False'  :"../data/train/CLASE II y IV, FAKE_COLOR/ID_REAL/ID",    
    'imagenes_ID_IV_True'   :"../data/train/CLASE II y IV, FAKE_COLOR/ID_FAKE_IV/ID_A", 


    'imagenes_ID_III_A_False':"../data/train/CLASE III, FAKE_BW/ID_REAL/ID", 
    'imagenes_ID_III_B_False':"../data/train/CLASE III, FAKE_BW/ID_REAL/ID", 

    'imagenes_ID_III_A_True':"../data/train/CLASE III, FAKE_BW/ID_FAKE_III/ID_A",
    'imagenes_ID_III_B_True':"../data/train/CLASE III, FAKE_BW/ID_FAKE_III/ID_B",  





}

#### Parámetros para modelos

In [3]:
#Parámetro CASO_0
threshold_identidad = 0.65
#Parámetro CASO_1
threshold_edad = 21


#### Creación instancias modelos individuales

In [4]:
Modelo_CLASE_0 = FaceVerifier(paths['predictor_path'], paths['face_rec_model_path'], threshold_identidad)
Modelo_CLASE_I = AgeChecker(paths['model_weights_path'], threshold_edad)
Modelo_CLASE_II_IV = Fake_II_IV_detector(paths['model_II_IV_path'])
Modelo_CLASE_III_A = Fake_III_A_detector(model_path=paths['model_III_A_path'], predictor_path=paths['predictor_path'])
Modelo_CLASE_III_B = Fake_III_B_detector(model_path=paths['model_III_B_path'], predictor_path=paths['predictor_path'])


Weights loaded successfully from path: ../modelos/weights.pt


In [5]:
def cargar_imagenes(carpeta, n=1000):
    imagenes = []
    for i in range(1, n + 1):
        imagen_path = os.path.join(f'{carpeta}_{i}.png')  
        imagenes.append(Image.open(imagen_path))
    return imagenes

def evaluar_modelo(imagenes_id, imagenes_val):
    y_verdadero = []
    y_predicho = []


    for i in tqdm(range(len(imagenes_id)), desc='Evaluando pares que deben dar False'):
        resultado = Modelo_CLASE_0.verify_faces(imagenes_id[i], imagenes_val[i])
        y_verdadero.append(False)
        y_predicho.append(resultado)

    for i in tqdm(range(len(imagenes_id)), desc='Evaluando pares que deben dar True'):
        j = (i + 1) % len(imagenes_id)  
        resultado = Modelo_CLASE_0.verify_faces(imagenes_id[i], imagenes_val[j])
        y_verdadero.append(True)
        y_predicho.append(resultado)


    accuracy = accuracy_score(y_verdadero, y_predicho)
    f1 = f1_score(y_verdadero, y_predicho)
    tn, fp, fn, tp = confusion_matrix(y_verdadero, y_predicho).ravel()
    error_tipo_i = fp / (fp + tn)  
    error_tipo_ii = fn / (fn + tp)  

    return accuracy, f1, error_tipo_i, error_tipo_ii

In [6]:
def evaluar_modelo_simple(carpeta_positivos, carpeta_negativos, modelo):

    def cargar_imagenes(carpeta, maximo=1000):
        imagenes = []
        archivos = os.listdir(carpeta)
        for filename in archivos[:maximo]:  
            if filename.endswith('.png'):
                imagen_path = os.path.join(carpeta, filename)
                imagenes.append(Image.open(imagen_path))
        return imagenes

    positivas = cargar_imagenes(carpeta_positivos)
    negativas = cargar_imagenes(carpeta_negativos)
    
    y_verdadero = []
    y_predicho = []
    
 
    for img in tqdm(positivas, desc='Evaluando imágenes positivas'):
        resultado = modelo(img)
        y_verdadero.append(True)
        y_predicho.append(resultado)

 
    for img in tqdm(negativas, desc='Evaluando imágenes negativas'):
        resultado = modelo(img)
        y_verdadero.append(False)
        y_predicho.append(resultado)


    accuracy = accuracy_score(y_verdadero, y_predicho)
    f1 = f1_score(y_verdadero, y_predicho, zero_division=1)
    tn, fp, fn, tp = confusion_matrix(y_verdadero, y_predicho).ravel()
    error_tipo_i = fp / (fp + tn) if (fp + tn) > 0 else 0  
    error_tipo_ii = fn / (fn + tp) if (fn + tp) > 0 else 0  

    return {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Error Tipo I': error_tipo_i * 100,
        'Error Tipo II': error_tipo_ii * 100
    }


#### Validación y cálculo de indicadores

In [7]:

imagenes_id = cargar_imagenes(paths['imagenes_ID'])
imagenes_val = cargar_imagenes(paths['imagenes_VAL'])

accuracy, f1, error_tipo_i, error_tipo_ii = evaluar_modelo(imagenes_id, imagenes_val)


print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Error Tipo I: {error_tipo_i * 100:.2f}%')
print(f'Error Tipo II: {error_tipo_ii * 100:.2f}%')

Evaluando pares que deben dar True: 100%|██████████| 1000/1000 [12:39<00:00,  1.32it/s]

Accuracy: 0.818
F1 Score: 0.7952755905511811
Error Tipo I: 7.10%
Error Tipo II: 29.30%


In [8]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_I_True'], paths['imagenes_ID_I_False'], Modelo_CLASE_I.is_minor)
print(resultados)

Evaluando imágenes positivas:  34%|███▍      | 21/61 [00:01<00:01, 20.71it/s]

Error al estimar la edad: '>' not supported between instances of 'NoneType' and 'float'


Evaluando imágenes negativas: 100%|██████████| 128/128 [00:06<00:00, 20.45it/s]

{'Accuracy': 0.8624338624338624, 'F1 Score': 0.7868852459016393, 'Error Tipo I': 10.15625, 'Error Tipo II': 21.311475409836063}


In [9]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_II_True'], paths['imagenes_ID_II_False'], Modelo_CLASE_II_IV.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:10<00:00, 92.79it/s]

{'Accuracy': 0.9885, 'F1 Score': 0.9886307464162135, 'Error Tipo I': 2.3, 'Error Tipo II': 0.0}


In [10]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_IV_True'], paths['imagenes_ID_IV_False'], Modelo_CLASE_II_IV.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:10<00:00, 91.22it/s]


{'Accuracy': 0.979, 'F1 Score': 0.9790419161676647, 'Error Tipo I': 2.3, 'Error Tipo II': 1.9}


In [11]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_III_A_True'], paths['imagenes_ID_III_A_False'], Modelo_CLASE_II_IV.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:10<00:00, 92.91it/s]


{'Accuracy': 0.5495, 'F1 Score': 0.2131004366812227, 'Error Tipo I': 2.3, 'Error Tipo II': 87.8}


In [12]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_III_B_True'], paths['imagenes_ID_III_B_False'], Modelo_CLASE_II_IV.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:10<00:00, 92.15it/s]

{'Accuracy': 0.5185, 'F1 Score': 0.11080332409972299, 'Error Tipo I': 2.3, 'Error Tipo II': 94.0}


In [13]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_III_A_True'], paths['imagenes_ID_III_A_False'], Modelo_CLASE_III_A.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [21:27<00:00,  1.29s/it]

{'Accuracy': 0.9875, 'F1 Score': 0.9874812218327491, 'Error Tipo I': 1.0999999999999999, 'Error Tipo II': 1.4000000000000001}


In [14]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_III_B_True'], paths['imagenes_ID_III_B_False'], Modelo_CLASE_III_A.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [12:07<00:00,  1.37it/s]

{'Accuracy': 0.5035, 'F1 Score': 0.03498542274052478, 'Error Tipo I': 1.0999999999999999, 'Error Tipo II': 98.2}


In [15]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_II_True'], paths['imagenes_ID_II_False'], Modelo_CLASE_III_A.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [10:55<00:00,  1.53it/s]

{'Accuracy': 0.9905, 'F1 Score': 0.9905142286570144, 'Error Tipo I': 1.0999999999999999, 'Error Tipo II': 0.8}


In [16]:
# En este escenario se prueba el modelo CLASE III con imágenes clase IV, las cuales tienen manchas de colores las que, en ocasiones, evitan que se detecte un rostro

resultados = evaluar_modelo_simple(paths['imagenes_ID_IV_True'], paths['imagenes_ID_IV_False'], Modelo_CLASE_III_A.predict)
print(resultados)

Evaluando imágenes positivas:   1%|          | 6/1000 [00:03<10:40,  1.55it/s]

Error processing image: No faces detected in the image.


Evaluando imágenes positivas:   1%|          | 10/1000 [00:06<10:35,  1.56it/s]

Error processing image: No faces detected in the image.


Evaluando imágenes positivas:   5%|▍         | 48/1000 [00:31<10:14,  1.55it/s]

...

Error processing image: No faces detected in the image.


Evaluando imágenes positivas: 100%|█████████▉| 996/1000 [10:51<00:02,  1.55it/s]

Error processing image: No faces detected in the image.


Evaluando imágenes negativas: 100%|██████████| 1000/1000 [10:54<00:00,  1.53it/s]

{'Accuracy': 0.516, 'F1 Score': 0.08159392789373814, 'Error Tipo I': 1.0999999999999999, 'Error Tipo II': 95.7}


In [17]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_III_B_True'], paths['imagenes_ID_III_B_False'], Modelo_CLASE_III_B.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:32<00:00, 30.86it/s]

{'Accuracy': 0.9865, 'F1 Score': 0.9863291139240506, 'Error Tipo I': 0.1, 'Error Tipo II': 2.6}


In [18]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_III_A_True'], paths['imagenes_ID_III_A_False'], Modelo_CLASE_III_B.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:32<00:00, 31.00it/s]

{'Accuracy': 0.5, 'F1 Score': 0.001996007984031936, 'Error Tipo I': 0.1, 'Error Tipo II': 99.9}


In [19]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_II_True'], paths['imagenes_ID_II_False'], Modelo_CLASE_III_B.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:33<00:00, 29.73it/s]

{'Accuracy': 0.5, 'F1 Score': 0.001996007984031936, 'Error Tipo I': 0.1, 'Error Tipo II': 99.9}


In [20]:
resultados = evaluar_modelo_simple(paths['imagenes_ID_IV_True'], paths['imagenes_ID_IV_False'], Modelo_CLASE_III_B.predict)
print(resultados)

Evaluando imágenes negativas: 100%|██████████| 1000/1000 [00:34<00:00, 29.09it/s]

{'Accuracy': 0.5005, 'F1 Score': 0.003988035892323031, 'Error Tipo I': 0.1, 'Error Tipo II': 99.8}
